Tests for retrieving data from PACS
--

Imports
--

In [ ]:
import logging
import configparser
import pandas as pd
from collections import namedtuple

from datetime import datetime as dt

from IPython.core import display as ICD

from pydicom.dataset import Dataset

from scripts.run_all import run_all
from scripts.retrieve_data_from_PACS import *

#from pynetdicom import debug_logger
#debug_logger()

pd.set_option('display.max_colwidth', -1)

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Set the logger of the pynetdicom module to ERROR, to avoid logs

In [ ]:
logging.getLogger('pynetdicom').setLevel(logging.ERROR)

Initialize the "config" object
--

In [ ]:
config = run_all()

Get all PT studies for a day
--

In [ ]:
df_studies = find_PT_studies_for_day(config, '20190806')
df_studies

Get all PT and CT series for the found studies and get their time ranges
--

In [ ]:
#for i_study in range(len(df_studies)):
for i_study in range(4,5):
    logging.debug('DataFrame row:\n' + str(df_studies.loc[i_study, :]))
    df_series = find_series_for_study(config, df_studies.loc[i_study, :])

    # go through each Series
    for i_series in range(len(df_series)):

        logging.debug('Series: ' + str(df_series.loc[i_series, :]))
        # fetch the information for the current serie
        row_info = fetch_info_for_series(config, df_series.loc[i_series, :])
        
        if row_info is None:
            logging.error('Skipping series {}: no data found.'.format(df_series.loc[i_series, 'Series Instance UID']))
            continue

        # copy the relevant parameters into the main DataFrame
        df_series.loc[i_series, 'start_time'] = row_info['start_time']
        df_series.loc[i_series, 'end_time']   = row_info['end_time']
        df_series.loc[i_series, 'machine']  = row_info['machine']
    
    # remove redundant series
    df_series = prunes_series_by_time_overlap(df_series)
    
    # create time ranges from the start/end times
    time_ranges = []
    for i_serie in range(len(df_series)):
        time_ranges.append('{}-{}'.format(df_series.loc[i_serie, 'start_time'],
                                          df_series.loc[i_serie, 'end_time']))
    
    # propagate back the time range and machine name information
    df_studies.loc[i_study, 'machines'] = ','.join(list(set(df_series['machine'])))
    df_studies.loc[i_study, 'time_ranges'] = ','.join(time_ranges)
  